In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np

In [5]:
# Load the rolling window sequence numpy array
sequence = np.load('/content/drive/MyDrive/data/rolling_window_sequences.npy')
print(f"Loaded rolling window sequences with shape: {sequence.shape}")

Loaded rolling window sequences with shape: (17631, 30, 66)


In [6]:
## Load the metadata
metadata_df = pd.read_csv('/content/drive/MyDrive/PrognosAI/milestone_1/Day_4/sequence_metadata.csv')
print(f"Loaded metadata with shape: {metadata_df.shape}")
metadata_df.shape

Loaded metadata with shape: (17631, 2)


(17631, 2)

In [7]:
## Calculate the RUL per sequence

# Find the max. cycle per engine in metadata
max_cycle_per_engine = metadata_df.groupby('engine_id')['cycle'].max().reset_index()
max_cycle_per_engine.columns = ['engine_id', 'max_cycle']
max_cycle_per_engine.head()

,engine_id,max_cycle
0,1,192.0
1,2,287.0
2,3,179.0
3,4,189.0
4,5,269.0


In [8]:
# Merge max cycle info with metadata
metadata_df = metadata_df.merge(max_cycle_per_engine, on='engine_id', how='left')
metadata_df.head()

,engine_id,cycle,max_cycle
0,1,31.0,192.0
1,1,32.0,192.0
2,1,33.0,192.0
3,1,34.0,192.0
4,1,35.0,192.0


In [9]:
# Compute RUL
metadata_df['RUL'] = metadata_df['max_cycle'] - metadata_df['cycle']

In [10]:
metadata_df.head()

,engine_id,cycle,max_cycle,RUL
0,1,31.0,192.0,161.0
1,1,32.0,192.0,160.0
2,1,33.0,192.0,159.0
3,1,34.0,192.0,158.0
4,1,35.0,192.0,157.0


In [11]:
# Drop 'max_cycle' column
metadata_df.drop(columns='max_cycle', inplace=True)

In [12]:
metadata_df.head()

,engine_id,cycle,RUL
0,1,31.0,161.0
1,1,32.0,160.0
2,1,33.0,159.0
3,1,34.0,158.0
4,1,35.0,157.0


In [13]:
## Validate by inspecting example engine(s)
engines_to_check = metadata_df['engine_id'].unique()[:3] # first 3 engines
for engine in engines_to_check:
  sample = metadata_df[metadata_df['engine_id'] == engine][['cycle', 'RUL']]
  print(f"\nEngine {engine} - Cycles and RULs:")
  print(sample.head(10))
  print(sample.tail(10))


Engine 1 - Cycles and RULs:
   cycle    RUL
0   31.0  161.0
1   32.0  160.0
2   33.0  159.0
3   34.0  158.0
4   35.0  157.0
5   36.0  156.0
6   37.0  155.0
7   38.0  154.0
8   39.0  153.0
9   40.0  152.0
     cycle  RUL
152  183.0  9.0
153  184.0  8.0
154  185.0  7.0
155  186.0  6.0
156  187.0  5.0
157  188.0  4.0
158  189.0  3.0
159  190.0  2.0
160  191.0  1.0
161  192.0  0.0

Engine 2 - Cycles and RULs:
     cycle    RUL
162   31.0  256.0
163   32.0  255.0
164   33.0  254.0
165   34.0  253.0
166   35.0  252.0
167   36.0  251.0
168   37.0  250.0
169   38.0  249.0
170   39.0  248.0
171   40.0  247.0
     cycle  RUL
409  278.0  9.0
410  279.0  8.0
411  280.0  7.0
412  281.0  6.0
413  282.0  5.0
414  283.0  4.0
415  284.0  3.0
416  285.0  2.0
417  286.0  1.0
418  287.0  0.0

Engine 3 - Cycles and RULs:
     cycle    RUL
419   31.0  148.0
420   32.0  147.0
421   33.0  146.0
422   34.0  145.0
423   35.0  144.0
424   36.0  143.0
425   37.0  142.0
426   38.0  141.0
427   39.0  140.0
428   4

In [15]:
## Save updated metadata with RUL target
metadata_df.to_csv('/content/drive/MyDrive/data/sequence_metadata_with_RUL.csv', index=False)